In [ ]:
# Obtiene una máscara para cada municipio del país.

!pip install bottleneck

import pandas as pd
import numpy as np

import xarray as xr

from multiprocessing import Pool

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/" 
path_data = path_drive + "data/"
f_mask = "municipios_mask.nc"
f_data = "data_0.csv"

# Abrimos los archivos a utilizar.
df = pd.read_csv( path_data + f_data )
mask = xr.open_dataset( path_drive + f_mask )

In [ ]:
# Procesa la variable para todos los años.
def proc_mean(year):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d + var_2 + "/mexico_" 
        + var_2 + "." + str(year) + ".nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        df_y = df[ df["Año"] == year ].copy()
        for i in range( df["Municipio"].max() + 1 ):
            # Obtenemos el promedio dentro de la máscara
            # del municipio.
            df_y.loc[ df["Municipio"] == i,
                var_1 ] = ( ds.where(mask
                .isel(municipio = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().mean()[0] )
        return df_y

# Procesa la variable para todos los años.
def proc_sum(year):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d + var_2 + "/mexico_" 
        + var_2 + "." + str(year) + ".nc" ) as ds:
        df_y = df[ df["Año"] == year ].copy()
        for i in range( df["Municipio"].max() + 1 ):
            # Obtenemos el promedio dentro de la máscara
            # del municipio.
            df_y.loc[ df["Municipio"] == i,
                var_1 ] = ( ds.where(mask
                .isel(municipio = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().sum()[0] )
        return df_y

In [ ]:
# Variables de CHIRTS y CHIRPS.
vars = ["Tmax", "Tmin", "Tmean", "HDD_mean", 
        "CDD_mean", "HDD_p10", 
        "CDD_p90", "Pre", "Pre_Tmean"]

# Variable a procesar, 0 - 8.
n = 7
var_1 =  df.columns[4:-9][n]
var_2 = vars[n]

if var_2 == "Pre":
    folder = "CHIRPS/"
elif var_2 in vars[0:3] : folder = "CHIRTS/"
else: folder = "vars/"

if var_2 in vars[3:7]:
    mask = mask.rename_vars(mask = "degree_day")
else:
    mask = mask.rename_vars(mask = var_2)

# Dirección de las variables.
path_d = path_drive + folder

# Años a procesar.
year = list(range(2010, 2017))

# Procesamos los años en paralelo.
pool = Pool()
if var_2 in vars[3:9]:
    df_r = pool.map(proc_sum, year)
else:
    df_r = pool.map(proc_mean, year)

# Concatenamos los resultados de cada año.
for f in df_r:
    df.loc[df["Año"] == f["Año"].max() ] = f

# Guardamos el resultado como un nuevo archivo.
df.to_csv(path_data + "data_" + str(n + 1)
     + ".csv", index = False)